In [2]:
import pandas as pd
import re
from shekar import Normalizer
import torch
 
df = pd.read_csv("samsung_comments_labeled_binary.csv")

 
normalizer = Normalizer()

def clean_text(text):
    if not isinstance(text, str):
        return ""
    # حذف لینک‌ها
    text = re.sub(r"http\S+|www\S+", " ", text)
    # حذف ایموجی‌ها و کاراکترهای غیر متنی
    text = re.sub(r"[^\w\s\u0600-\u06FF]", " ", text)
    # حذف اعداد
    text = re.sub(r"\d+", " ", text)
    # نرمال‌سازی حروف فارسی (ي→ی، ك→ک و ...)
    text = normalizer.normalize(text)
    # حذف فاصله‌های اضافی
    text = re.sub(r"\s+", " ", text).strip()
    return text

# --- Apply cleaning to every row ---
df["clean_body"] = df["body"].apply(clean_text)
print(df.head())


                                                body    created_at  rate  \
0  بعد از ۴ ماه استفاده میگم، از همه نظر راضیم از...  13 آبان 1404   5.0   
1    سلام تشکر از دی جی کالا عزیز برای برادرم خریدم   13 آبان 1404   4.0   
2  کیفیت دور بین عالی هست و برای کار روزمره خوبه ...  13 آبان 1404   4.0   
3  سلام همه چیش اوکی هست ولی همین که وقتی روی میز...  13 آبان 1404   4.0   
4  گوشی بسیار با صفحه نمایش عالی ..ولی عملکرد خوب...  13 آبان 1404   0.0   

                                        product_name         user_name label  \
0  گوشی موبایل سامسونگ مدل S24 FE دو سیم کارت ظرف...   کاربر دیجی‌کالا  مثبت   
1  گوشی موبایل سامسونگ مدل S24 FE دو سیم کارت ظرف...  محمدعلی مختارپور  مثبت   
2  گوشی موبایل سامسونگ مدل S24 FE دو سیم کارت ظرف...  حامد اشکوری ثالث  مثبت   
3  گوشی موبایل سامسونگ مدل S24 FE دو سیم کارت ظرف...   کاربر دیجی‌کالا  منفی   
4  گوشی موبایل سامسونگ مدل S24 FE دو سیم کارت ظرف...   کاربر دیجی‌کالا  مثبت   

   positive_prob                                         clean

In [3]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
from tqdm import tqdm
import os
 
model_name = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()


def get_parsbert_embedding(text, tokenizer, model, device):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    )
    inputs = {k:v.to(device) for k,v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    
    embedding = outputs.last_hidden_state[:,0,:].squeeze().cpu().numpy()
    return embedding
 
def batch_embeddings(texts, tokenizer, model, device, batch_size=16):
    all_embeddings = [] 
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding batches"):
        batch_texts = texts[i:i+batch_size]
        batch_emb = [get_parsbert_embedding(t, tokenizer, model, device) for t in batch_texts]
        all_embeddings.extend(batch_emb)
    return all_embeddings


texts = df["clean_body"].tolist()

if os.path.exists("embeddings.npy"):
    print("Loading saved embeddings...")
    embeddings = np.load("embeddings.npy", allow_pickle=True)
    embeddings = list(embeddings) 
else:
    print("Generating embeddings...") 
    embeddings = batch_embeddings(texts, tokenizer, model, device, batch_size=16) 
    np.save("embeddings.npy", np.stack(embeddings))
 
df["embedding"] = embeddings


print("Embedding ها ساخته و ذخیره شدند ")

Loading saved embeddings...
Embedding ها ساخته و ذخیره شدند 


In [4]:
X = np.stack(df['embedding'].values)  # شکل: (9531, 768)

print("نوع داده:", type(X))
print("شکل داده:", X.shape)


نوع داده: <class 'numpy.ndarray'>
شکل داده: (9530, 768)


In [5]:
y = df['label'].map({'مثبت': 1, 'منفی': 0})
print(y)

0       1
1       1
2       1
3       0
4       1
       ..
9525    1
9526    1
9527    1
9528    1
9529    0
Name: label, Length: 9530, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler() 
X_train = scaler.fit_transform(X) 

dbscan = DBSCAN(eps=3, min_samples=10, metric='euclidean', n_jobs=-1)
labels = dbscan.fit_predict(X_train)

print(labels)
df["Means"] = labels

print(df[['body', 'Means']].head())

[ 0  0 -1 ...  0  0  0]
                                                body  Means
0  بعد از ۴ ماه استفاده میگم، از همه نظر راضیم از...      0
1    سلام تشکر از دی جی کالا عزیز برای برادرم خریدم       0
2  کیفیت دور بین عالی هست و برای کار روزمره خوبه ...     -1
3  سلام همه چیش اوکی هست ولی همین که وقتی روی میز...      0
4  گوشی بسیار با صفحه نمایش عالی ..ولی عملکرد خوب...      0


In [8]:
df[['body','Means','user_name','label']].to_csv('body_label.csv', index=False, encoding='utf-8-sig')